In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!wget http://cicresearch.ca/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip

--2025-05-13 13:26:07--  http://cicresearch.ca/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip
Resolving cicresearch.ca (cicresearch.ca)... 205.174.165.80
Connecting to cicresearch.ca (cicresearch.ca)|205.174.165.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235102953 (224M) [application/zip]
Saving to: ‘MachineLearningCSV.zip’

MachineLearningCSV. 100%[===================>] 224.21M  4.92MB/s    in 28s     

2025-05-13 13:26:36 (7.96 MB/s) - ‘MachineLearningCSV.zip’ saved [235102953/235102953]



In [3]:
!unzip MachineLearningCSV.zip -d MachineLearningCSV

Archive:  MachineLearningCSV.zip
   creating: MachineLearningCSV/MachineLearningCVE/
  inflating: MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv  
  inflating: MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv  


In [4]:
!pip install scikit-learn

In [5]:
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [6]:
import gc

In [7]:
MODEL_PATH = 'phishing_detector.pkl'

def load_unsw_dataset():
    files = [
        "MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv",
        "MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
        "MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv",
        "MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv", 
        "MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
        "MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
        # "MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv",
        # "MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv"
    ]
    dfs = []
    for f in files:
        if os.path.exists(f):
            dfs.append(pd.read_csv(f))
        else:
            print(f"Missing file: {f}")
    
    if not dfs:
        print("No data files found. Please check the file paths.")
        return None
    
    data = pd.concat(dfs, ignore_index=True)
    data.columns = data.columns.str.strip()
    cols = ['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
       'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes','Label']
    
    # Get only columns that exist in the data
    valid_cols = [col for col in cols if col in data.columns]
    if set(valid_cols) != set(cols):
        missing = set(cols) - set(valid_cols)
        print(f"Warning: Missing columns: {missing}")
    
    data_1 = data[valid_cols]
    return data_1

data = load_unsw_dataset()

In [8]:
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,0,0,0,0,0,1,6,1,6,BENIGN
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,0,0,0,0,0,11,172,5,326,BENIGN
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,0,0,0,0,0,10,3150,6,3150,BENIGN
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,0,0,0,0,0,17,3452,12,6660,BENIGN
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,0,0,0,0,0,9,3150,6,3152,BENIGN


In [9]:
for column,null_vals in data.isna().sum().items():
    print(column,null_vals)

Destination Port 0
Flow Duration 0
Total Fwd Packets 0
Total Backward Packets 0
Total Length of Fwd Packets 0
Total Length of Bwd Packets 0
Fwd Packet Length Max 0
Fwd Packet Length Min 0
Fwd Packet Length Mean 0
Fwd Packet Length Std 0
Bwd Packet Length Max 0
Bwd Packet Length Min 0
Bwd Packet Length Mean 0
Bwd Packet Length Std 0
Flow Bytes/s 1153
Flow Packets/s 0
Flow IAT Mean 0
Flow IAT Std 0
Flow IAT Max 0
Flow IAT Min 0
Fwd IAT Total 0
Fwd IAT Mean 0
Fwd IAT Std 0
Fwd IAT Max 0
Fwd IAT Min 0
Bwd IAT Total 0
Bwd IAT Mean 0
Bwd IAT Std 0
Bwd IAT Max 0
Bwd IAT Min 0
Fwd PSH Flags 0
Bwd PSH Flags 0
Fwd URG Flags 0
Bwd URG Flags 0
Fwd Header Length 0
Bwd Header Length 0
Fwd Packets/s 0
Bwd Packets/s 0
Min Packet Length 0
Max Packet Length 0
Packet Length Mean 0
Packet Length Std 0
Packet Length Variance 0
FIN Flag Count 0
SYN Flag Count 0
RST Flag Count 0
PSH Flag Count 0
ACK Flag Count 0
URG Flag Count 0
CWE Flag Count 0
ECE Flag Count 0
Down/Up Ratio 0
Average Packet Size 0
Avg Fwd 

In [10]:
data.shape

(2159089, 67)

In [11]:
import numpy as np

data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [12]:
data = data.dropna()
data.shape

(2156520, 67)

In [13]:
float_cols = data.select_dtypes(include=['float64', 'float32']).columns
data[float_cols] = data[float_cols].round(3)

In [14]:
data['Label'].value_counts()

Label
BENIGN                        1741821
DoS Hulk                       230124
PortScan                       158804
DoS GoldenEye                   10293
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack   Brute Force         1507
Web Attack   XSS                  652
Infiltration                       36
Web Attack   Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [15]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X = data.drop(columns = ["Label"],axis = 1)
y = data["Label"]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [17]:
del X,y
gc.collect()

0

In [18]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [19]:
del smote,X_train,y_train
gc.collect()

0

In [20]:
import pandas as pd
import numpy as np
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [21]:
le = LabelEncoder()
y_train = le.fit_transform(y_resampled)

In [22]:
y_test = le.transform(y_test)

In [23]:
# Print the mapping
for i, label in enumerate(le.classes_):
    print(f"{label} → {i}")

BENIGN → 0
Bot → 1
DoS GoldenEye → 2
DoS Hulk → 3
DoS Slowhttptest → 4
DoS slowloris → 5
Heartbleed → 6
Infiltration → 7
PortScan → 8
Web Attack   Brute Force → 9
Web Attack   Sql Injection → 10
Web Attack   XSS → 11


In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_resampled)

In [25]:
del X_resampled,y_resampled
gc.collect()

0

In [31]:
!pip install xgboost

In [32]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score

# Train the model
xgb_model = xgb.XGBClassifier(
    n_estimators=100,  # Number of boosting rounds
    learning_rate=0.1,  # Step size for each boosting round
    max_depth=6,  # Depth of each tree
    random_state=42,  # For reproducibility
    n_jobs=-1  # Use all CPU cores
)



In [33]:
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.79852540203661453


In [34]:
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

Classification Report:
                             precision    recall  f1-score   support

                    BENIGN       0.87      0.92      0.89    435455
                       Bot       0.51      0.42      0.46       489
             DoS GoldenEye       0.60      0.58      0.59      2573
                  DoS Hulk       0.76      0.72      0.74     57531
          DoS Slowhttptest       0.55      0.49      0.52      1375
             DoS slowloris       0.53      0.46      0.49      1449
                Heartbleed       1.00      0.67      0.80         3
              Infiltration       0.70      0.78      0.74         9
                  PortScan       0.75      0.69      0.72     39701
  Web Attack   Brute Force       0.57      0.54      0.55       377
Web Attack   Sql Injection       0.62      0.60      0.61         5
          Web Attack   XSS       0.59      0.52      0.55       163

                  accuracy                           0.79    539130
                 macro

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
import joblib
joblib.dump(xgb_model, 'xgboost_model.pkl')
# Save the LabelEncoder
joblib.dump(le, 'label_encoder.pkl')

# # Save the scaler (if you used one, e.g., StandardScaler)
joblib.dump(scaler, 'scaler.pkl')  # Replace 'scaler' with your actual scaler variable

['scaler.pkl']